# ArrayManager Example Notebook

In [2]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import numpy as np
import dask.array as da

In [3]:
# Step 1: Create a synthetic 5D image (T, C, Z, Y, X)
t, c, z, y, x = 10, 2, 512, 2048, 2048
np_data = np.random.randint(0, 5000, size=(t, c, z, y, x), dtype=np.uint16)
dask_data = da.from_array(np_data, chunks=(1, 1, 1, 1024, 1024))

In [4]:
# Step 2: Define metadata
metadata = {
            "size": [dask_data.shape],
            "scales": [(4.0, 0.65, 0.65)],
            "units": ("micrometer", "micrometer", "micrometer"),
            "time_increment": 1.0,
            "time_increment_unit": "s",
            "channel_names": ["Red", "Green"],
            "channel_colors": [16711680, 255],
            "dtype": "uint16",
            "axes": "tczyx",
        }

In [5]:
# Step 3: Instantiate ArrayManager
dataset = mm.ArrayManager(dask_data, metadata)
print("Loaded data shape:", dataset.data[0].shape)

Loaded data shape: (10, 2, 512, 2048, 2048)


In [6]:
# Step 4: Build pyramid (starting from base level)
dataset.build_pyramid(num_levels=3)
print("New pyramid levels:", [arr.shape for arr in dataset.data])

Requested start level 0
Resolution layer already available
New pyramid levels: [(10, 2, 512, 2048, 2048), (10, 2, 256, 1024, 1024), (10, 2, 128, 512, 512)]


In [7]:
# # Step 5 (optional): Visualize
# viewer = dataset.visualize()

In [8]:
# Reorder channels
dataset.reorder_channels([1,0])
# Confirm channels reodered
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")

✅ Channels reordered to [1, 0]
SIZE: [(10, 2, 512, 2048, 2048), (10, 2, 256, 1024, 1024), (10, 2, 128, 512, 512)]
SCALES: [(4.0, 0.65, 0.65), (8.0, 1.3, 1.3), (16.0, 2.6, 2.6)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 1.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['Green', 'Red']
CHANNEL_COLORS: [255, 16711680]
DTYPE: uint16
AXES: tczyx
PLANE_FILES: None


In [9]:
# Change metadata
dataset.update_metadata({
    "channel_names":["Blue", "Red"],
    "time_increment_unit": "m",
})
# Confirm metadata changed
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")

✅ Updated metadata entry 'channel_names'
✅ Updated metadata entry 'time_increment_unit'
SIZE: [(10, 2, 512, 2048, 2048), (10, 2, 256, 1024, 1024), (10, 2, 128, 512, 512)]
SCALES: [(4.0, 0.65, 0.65), (8.0, 1.3, 1.3), (16.0, 2.6, 2.6)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 1.0
TIME_INCREMENT_UNIT: m
CHANNEL_NAMES: ['Blue', 'Red']
CHANNEL_COLORS: [255, 16711680]
DTYPE: uint16
AXES: tczyx
PLANE_FILES: None


In [ ]:
# Write the data to disk in serial but with some compression
import time
start = time.time()
dataset.write(
    "/g/mif/people/gritti/code/pymif_test_data/test_array_write_serial_GZIP.zarr",
    compressor = "GZIP",
    parallelize = False
)
print(time.time()-start)

False


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f83a1133520>>
Traceback (most recent call last):
  File "/home/gritti/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [12]:
dataset_zarr = mm.ZarrManager("/g/mif/people/gritti/code/pymif_test_data/test_array_write_serial.zarr")

In [13]:
# Check that writing happens in parallel
import time
start = time.time()
dataset_zarr.write(
    "/g/mif/people/gritti/code/pymif_test_data/test_array_write_resaved_parallel.zarr",
    compressor = None,
    parallelize = True
)
print(time.time()-start)

True


/opt/miniconda3/envs/pymif/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36141 instead
  warnings.warn(


Dask dashboard: http://127.0.0.1:36141/status


2025-06-02 10:04:42,696 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 1a3b839aa410129833543c13753790b5 initialized by task ('rechunk-merge-rechunk-transfer-7f5f69f48bd7b1a1af3bec988585a951', 9, 1, 8, 1, 1, 9, 1, 8, 1, 1) executed on worker tcp://127.0.0.1:45825
2025-06-02 10:04:42,708 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle bbfa046ba90f842537d311fb436a0460 initialized by task ('rechunk-merge-rechunk-transfer-7f5f69f48bd7b1a1af3bec988585a951', 9, 1, 9, 0, 0, 9, 1, 9, 0, 0) executed on worker tcp://127.0.0.1:45825
2025-06-02 10:04:42,716 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 9a5ab10f5e05fb88fb45c8eebdf52c44 initialized by task ('rechunk-merge-rechunk-transfer-7f5f69f48bd7b1a1af3bec988585a951', 9, 1, 8, 0, 1, 9, 1, 8, 0, 1) executed on worker tcp://127.0.0.1:38285
2025-06-02 10:04:42,724 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle fa323450f6906be1ac40bd8bf5a7ccb8 initialized by task ('rechunk-merge-rechunk-transfer-7f

337.7843463420868
